<a href="https://colab.research.google.com/github/YeonKang/Tensorflow-with-Colab/blob/master/Extra__tf_function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import traceback
import contextlib

#Helper function for error output
@contextlib.contextmanager
def assert_raises(error_class):
  try:
    yield
  except error_class as e:
    print('Throwing the expected exception \n  {}:'.format(error_class))
    traceback.print_exc(limit=2)
  except Exception as e:
    raise e
  else:
    raise Exception('Expected {}, but no error occurred'.format(
        error_class))

**calcultate gradient descent with tf.function**

In [2]:
@tf.function
def add(a, b):
  return a + b

add(tf.ones([2, 2]), tf.ones([2, 2]))  #  [[2., 2.], [2., 2.]]

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2., 2.],
       [2., 2.]], dtype=float32)>

In [3]:
v = tf.Variable(1.0)
with tf.GradientTape() as tape:
  result = add(v, 1.0)
tape.gradient(result, v)

<tf.Tensor: shape=(), dtype=float32, numpy=1.0>

**use tf.function in another function**

In [4]:
@tf.function
def dense_layer(x, w, b):
  return add(tf.matmul(x, w), b)

In [5]:
dense_layer(tf.ones([3, 2]), tf.ones([2, 2]), tf.ones([2]))

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[3., 3.],
       [3., 3.],
       [3., 3.]], dtype=float32)>

**Compare tf.function and eager execution for a large amount of computation**

In [6]:
import timeit
conv_layer = tf.keras.layers.Conv2D(100, 3)

@tf.function
def conv_fn(image):
  return conv_layer(image)

image = tf.zeros([1, 200, 200, 100])

conv_layer(image); conv_fn(image)
print("tf.function convolution:", timeit.timeit(lambda: conv_fn(image), number=10))
print("eager execution convolution:", timeit.timeit(lambda: conv_layer(image), number=10))
print("There is no significant difference in the speed of the convolution operation.")

tf.function convolution: 1.3088361030000328
eager execution convolution: 1.3439397289999988
There is no significant difference in the speed of the convolution operation.


**Tracing with tf.function**

In [7]:
@tf.function
def double(a):
  print("Tracing:", a)
  return a + a

print(double(tf.constant(1)))
print()
print(double(tf.constant(1.1)))
print()
print(double(tf.constant("a")))
print()

Tracing: Tensor("a:0", shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)

Tracing: Tensor("a:0", shape=(), dtype=float32)
tf.Tensor(2.2, shape=(), dtype=float32)

Tracing: Tensor("a:0", shape=(), dtype=string)
tf.Tensor(b'aa', shape=(), dtype=string)



In [8]:
def f():
  print('Tracing')
  tf.print('run')

tf.function(f)()
tf.function(f)()

Tracing
run
Tracing
run


**variable with tf.function**

In [9]:
@tf.function
def f(x):
  v = tf.Variable(1.0)
  v.assign_add(x)
  return v

with assert_raises(ValueError):
  f(1.0)

Throwing the expected exception 
  <class 'ValueError'>:


Traceback (most recent call last):
  File "<ipython-input-1-4081a9e711c4>", line 9, in assert_raises
    yield
  File "<ipython-input-9-73e410646579>", line 8, in <module>
    f(1.0)
ValueError: in user code:

    <ipython-input-9-73e410646579>:3 f  *
        v = tf.Variable(1.0)
    /usr/local/lib/python3.7/dist-packages/tensorflow/python/ops/variables.py:262 __call__  **
        return cls._variable_v2_call(*args, **kwargs)
    /usr/local/lib/python3.7/dist-packages/tensorflow/python/ops/variables.py:256 _variable_v2_call
        shape=shape)
    /usr/local/lib/python3.7/dist-packages/tensorflow/python/ops/variables.py:67 getter
        return captured_getter(captured_previous, **kwargs)
    /usr/local/lib/python3.7/dist-packages/tensorflow/python/eager/def_function.py:731 invalid_creator_scope
        "tf.function-decorated function tried to create "

    ValueError: tf.function-decorated function tried to create variables on non-first call.



In [10]:
v = tf.Variable(1.0)

@tf.function
def f(x):
  return v.assign_add(x)

print(f(1.0))  # 2.0
print(f(2.0))  # 4.0

tf.Tensor(2.0, shape=(), dtype=float32)
tf.Tensor(4.0, shape=(), dtype=float32)


**Generate variables only the first time a function is called**

In [11]:
class C:
  pass

obj = C()
obj.v = None

@tf.function
def g(x):
  if obj.v is None:
    obj.v = tf.Variable(1.0)
  return obj.v.assign_add(x)

print(g(1.0))  # 2.0
print(g(2.0))  # 4.0

tf.Tensor(2.0, shape=(), dtype=float32)
tf.Tensor(4.0, shape=(), dtype=float32)


**AutoGraph Transform**

In [12]:
@tf.function
def f(x):
  while tf.reduce_sum(x) > 1:
    tf.print(x)
    x = tf.tanh(x)
  return x

f(tf.random.uniform([5]))

[0.171340346 0.569562316 0.432840586 0.880022526 0.629393935]
[0.169683099 0.515037775 0.407692462 0.706430674 0.55763483]
[0.168073103 0.473861068 0.386511624 0.608433783 0.50622046]
[0.166508153 0.441313714 0.368349016 0.543023705 0.46699515]
[0.164986193 0.414732754 0.352546722 0.495273381 0.435768247]
[0.163505301 0.392483801 0.338632166 0.458391845 0.41013059]
[0.162063658 0.373499483 0.326255679 0.428772599 0.388583511]
[0.160659537 0.357048869 0.315152347 0.404295 0.370138437]
[0.159291372 0.342611939 0.305117071 0.383617908 0.354112774]
[0.157957628 0.329807192 0.295988411 0.365845293 0.340017885]
[0.156656876 0.318347484 0.287637144 0.350352317 0.32749337]
[0.155387789 0.308011949 0.279958576 0.336687922 0.316266626]
[0.1541491 0.298627436 0.272866696 0.32451728 0.30612728]
[0.152939618 0.29005602 0.266290277 0.313585728 0.296909839]
[0.151758209 0.282186359 0.260169566 0.303695619 0.28848213]
[0.150603801 0.274927378 0.254454106 0.294690937 0.280737132]
[0.149475396 0.2682037

<tf.Tensor: shape=(5,), dtype=float32, numpy=
array([0.13765493, 0.21345471, 0.20346244, 0.22228569, 0.21613117],
      dtype=float32)>

In [13]:
print(tf.autograph.to_code(f.python_function))

def tf__f(x):
    with ag__.FunctionScope('f', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        do_return = False
        retval_ = ag__.UndefinedReturnValue()

        def get_state():
            return (x,)

        def set_state(vars_):
            nonlocal x
            (x,) = vars_

        def loop_body():
            nonlocal x
            ag__.converted_call(ag__.ld(tf).print, (ag__.ld(x),), None, fscope)
            x = ag__.converted_call(ag__.ld(tf).tanh, (ag__.ld(x),), None, fscope)

        def loop_test():
            return (ag__.converted_call(ag__.ld(tf).reduce_sum, (ag__.ld(x),), None, fscope) > 1)
        ag__.while_stmt(loop_test, loop_body, get_state, set_state, ('x',), {})
        try:
            do_return = True
            retval_ = ag__.ld(x)
        except:
            do_return = False
            raise
        return fscope.ret(retval_, do_return)



**if condition with tf.function**

In [14]:
@tf.function
def fizzbuzz(n):
  for i in tf.range(1, n + 1):
    print('loop tracing')
    if i % 15 == 0:
      print('fizzbuzz branch tracing')
      tf.print('fizzbuzz')
    elif i % 3 == 0:
      print('fizz branch tracing')
      tf.print('fizz')
    elif i % 5 == 0:
      print('buzz branch tracing')
      tf.print('buzz')
    else:
      print('default branch tracing')
      tf.print(i)

fizzbuzz(tf.constant(5))
fizzbuzz(tf.constant(20))

loop tracing
fizzbuzz branch tracing
fizz branch tracing
buzz branch tracing
default branch tracing
1
2
fizz
4
buzz
1
2
fizz
4
buzz
fizz
7
8
fizz
buzz
11
fizz
13
14
fizzbuzz
16
17
fizz
19
buzz


**tf.TensorArray (To accumulate values over and over again)**

In [15]:
batch_size = 2
seq_len = 3
feature_size = 4

def rnn_step(inp, state):
  return inp + state

@tf.function
def dynamic_rnn(rnn_step, input_data, initial_state):
  # [batch, time, features] -> [time, batch, features]
  input_data = tf.transpose(input_data, [1, 0, 2])
  max_seq_len = input_data.shape[0]

  states = tf.TensorArray(tf.float32, size=max_seq_len)
  state = initial_state
  for i in tf.range(max_seq_len):
    state = rnn_step(input_data[i], state)
    states = states.write(i, state)
  return tf.transpose(states.stack(), [1, 0, 2])
  
dynamic_rnn(rnn_step,
            tf.random.uniform([batch_size, seq_len, feature_size]),
            tf.zeros([batch_size, feature_size]))

<tf.Tensor: shape=(2, 3, 4), dtype=float32, numpy=
array([[[0.9214865 , 0.6157757 , 0.60765004, 0.71412957],
        [1.6668649 , 1.3030404 , 1.499007  , 1.0704623 ],
        [1.7947794 , 2.1624327 , 1.5400409 , 1.2879518 ]],

       [[0.8468505 , 0.47634947, 0.5120218 , 0.9935986 ],
        [1.7853754 , 0.63917863, 1.2135186 , 1.4873818 ],
        [2.1632028 , 1.482266  , 1.4630289 , 1.9394392 ]]], dtype=float32)>